# Exploratory Data Analysis (EDA)  
## ClinVar Conflicting Variant Classification

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



df = pd.read_csv('../data/raw/clinvar_conflicting.csv',
                 dtype={'CHROM': str})


In [ ]:
# Vista general del conjunt de dades
print("Forma")
print(df.shape)

print("Tipus de dades")
df.info()



In [ ]:
df.head(15)


## Descripció de variables (resum)


| Variable (tipus) | Descripció | Variable (tipus) | Descripció | Variable (tipus) | Descripció |
|---|---|---|---|---|---|
| **CHROM** (cat) | Cromosoma | **POS** (num) | Posició dins cromosoma. | **REF** (cat) | Al·lel de referència. |
| **ALT** (cat) | Al·lel alternatiu. | **Allele** (cat) | Al·lel reportat. | **CLASS** (bin) | **Target**: 1 = conflictiva, 0 = consistent. |
| **AF_ESP** (num) | Freqüència al·lèlica (GO-ESP). | **AF_EXAC** (num) | Freqüència al·lèlica (ExAC). | **AF_TGP** (num) | Freqüència al·lèlica (1000G). |
| **CLNDISDB** (txt/cat↑) | IDs de malalties (tag-value). | **CLNDISDBINCL** (txt) | Idem “included variant”. | **CLNDN** (txt/cat) | Nom preferit de malaltia. |
| **CLNDNINCL** (txt) | Idem “included variant”. | **CLNHGVS** (txt) | Expressió HGVS. | **CLNSIGINCL** (txt) | Significància clínica “included variant”. |
| **CLNVC** (cat) | Tipus de variant (SNV, deletion, insertion...). | **CLNVI** (txt/cat) | Info extra / refs de variant. | **MC** (txt/cat) | Conseqüència molecular |
| **ORIGIN** (int/cat) | Origen al·lel. | **SSR** (num) | Regió/repetició. | **DISTANCE** (num) | Distància a feature. |
| **Consequence** (cat) | Conseqüències VEP. | **IMPACT** (cat) | Impacte funcional (HIGH/MODERATE/LOW/MODIFIER). | **SYMBOL** (cat) | Gen. |
| **Feature_type** (cat) | Tipus de feature. | **Feature** (cat) | Identificador de feature. | **BIOTYPE** (cat) | Biotip. |
| **EXON** (txt) | Exó en format `x/y`. | **INTRON** (txt) | Intró en format `x/y`. | **cDNA_position** (txt) | Posició relativa a cDNA. |
| **CDS_position** (txt) | Posició relativa a CDS. | **Protein_position** (txt) | Posició relativa a proteïna. | **Amino_acids** (txt) | Canvi AA. |
| **Codons** (txt) | Canvi de codó. | **STRAND** (cat) | Cadena (+1 / -1). | **BAM_EDIT** (cat) | Estat/flag de camp categòric. |
| **SIFT** (cat) | Predicció SIFT. | **PolyPhen** (cat) | Predicció PolyPhen. | **MOTIF_NAME** (txt) | Nom del motiu. |
| **MOTIF_POS** (num) | Posició en motiu. | **HIGH_INF_POS** (txt) | Flag/posició. | **MOTIF_SCORE_CHANGE** (num) | Canvi score del motiu . |
| **LoFtool** (num) | Tolerància a LoF (loss-of-function). | **CADD_PHRED** (num) | Score CADD. | **CADD_RAW** (num) | Score CADD. |
| **BLOSUM62** (num) | Score BLOSUM62 del canvi AA. |  |  |  |  |

## Distribucio de la variable objectiu (CLASS)

El grafic mostra un desbalanceig on **CLASS=0** es majoritaria (~75%) i **CLASS=1** representa ~25%.


In [ ]:
# Distribucio de la variable objectiu
counts = df["CLASS"].value_counts().sort_index()
pct = df["CLASS"].value_counts(normalize=True).sort_index() * 100

plt.figure(figsize=(5, 4))
plt.bar(counts.index.astype(str), counts.values)
plt.title("Target distribution: CLASS")
plt.xlabel("CLASS")
plt.ylabel("Count")
for i, v in enumerate(counts.values):
    plt.text(i, v, f"{v}\n({pct.iloc[i]:.1f}%)", ha="center", va="bottom", fontsize=9)
plt.tight_layout()
plt.show()



## Valors nuls

L’anàlisi dels valors nuls permet avaluar la qualitat i completitud del dataset. El gràfic de percentatge de valors absents mostra que diverses variables presenten una proporció molt elevada de missing values, incloent columnes amb pràcticament absència total d’informació. Aquest patró suggereix que algunes variables poden no ser útils per al modelatge i haurien de ser considerades per eliminació en la fase de preprocessament. En canvi, altres variables amb percentatges intermedis de missing, com SIFT o PolyPhen, podrien contenir informació rellevant quan estan presents, i la seva absència podria fins i tot tenir valor informatiu. Per aquest motiu, es defineix un llindar del 95% de missing per identificar variables candidates a ser descartades en etapes posteriors.


In [ ]:
# Valors nuls globals
missing_tbl = (
    df.isna()
      .mean()
      .mul(100)
      .sort_values(ascending=False)
      .to_frame("missing_%")
)
missing_tbl["dtype"] = df.dtypes.astype(str)
missing_tbl["nunique"] = df.nunique(dropna=True)
missing_tbl["n_missing"] = df.isna().sum()

# Principals columnes amb mes valors nuls
top_n = 20
top_missing = missing_tbl[missing_tbl["missing_%"] > 0].head(top_n).copy()

plt.figure(figsize=(10, 6))
plt.barh(top_missing.index[::-1], top_missing["missing_%"][::-1])
plt.xlabel("Percentatge de valors nuls (%)")
plt.title(f"Top {top_n} columnes amb més valors nuls")
plt.tight_layout()
plt.show()

# Columnes que superen el llindar
DROP_MISSING_THRESHOLD = 95.0
cols_drop_missing = missing_tbl.index[missing_tbl["missing_%"] > DROP_MISSING_THRESHOLD].tolist()
print(f"Columnes amb >{DROP_MISSING_THRESHOLD}% valors nuls: {len(cols_drop_missing)}")
print(cols_drop_missing[:30])



## Valors nuls per Classe (CLASS)

L’anàlisi del percentatge de valors absents segmentat per classe mostra que, en general, els valors nuls son similars entre **CLASS=0** i **CLASS=1**. Tot i que s’observen petites diferències en algunes variables, aquestes són moderades i no indiquen un patró clarament diferenciat entre classes. Per tant, la presència de valors nuls no sembla estar fortament associada amb la variable objectiu. 

In [ ]:
# Valors nuls per classe
target = "CLASS"

# Percentatge de valors nuls per columna i classe
missing_by_cls = df.groupby(target).apply(lambda g: g.isna().mean() * 100).T
missing_by_cls = missing_by_cls.drop(index=target, errors="ignore")

# Diferencia absoluta de valors nuls entre classes
missing_by_cls["diff_abs"] = (missing_by_cls[0] - missing_by_cls[1]).abs()

top_n = 15
top = missing_by_cls.sort_values("diff_abs", ascending=False).head(top_n)

x = np.arange(len(top.index))
width = 0.38

plt.figure(figsize=(11, 6))
plt.bar(x - width/2, top[0].values, width, label="CLASS=0")
plt.bar(x + width/2, top[1].values, width, label="CLASS=1")

plt.xticks(x, top.index, rotation=45, ha="right")
plt.ylabel("Percentatge de valors nuls (%)")
plt.title(f"Top {top_n} característiques amb diferència més gran de valors nuls per CLASS")
plt.legend()
plt.tight_layout()
plt.show()


## Variables Genòmiques Bàsiques: CHROM, POS, REF i ALT

Les variables genòmiques bàsiques descriuen la localització i la naturalesa molecular de cada variant. **CHROM** identifica el cromosoma on es troba la variant, **POS** indica la seva posició exacta dins del cromosoma, mentre que **REF** i **ALT** representen respectivament l’al·lel de referència i l’al·lel alternatiu observat. Aquestes variables constitueixen la definició única de cada variant i són essencials per comprendre la seva distribució i possible relació amb la presència de conflictes en la classificació.

### Distribució de CLASS per Cromosoma

El gràfic mostra el nombre absolut de variants consistents i conflictives per cromosoma. S’observa que alguns cromosomes, com el cromosoma 2, presenten un volum total de variants considerablement superior, tant en variants consistents com conflictives. Aquest resultat reflecteix principalment el nombre total de variants registrades en cada cromosoma i no necessàriament una major propensió al conflicte.


In [ ]:
# Variables genòmiques bàsiques

basic_cols = ["CHROM", "POS", "REF", "ALT"]
for c in basic_cols:
    assert c in df.columns, f"Falta columna: {c}"

# Recompte de classes per cromosoma
plt.figure(figsize=(10, 10))
sns.countplot(x="CLASS", data=df, hue="CHROM", palette="icefire")
plt.title("Distribució de CLASS per cromosoma")
plt.xlabel("CLASS")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

### Taxa de Conflicte per Cromosoma

El gràfic mostra la proporció de variants conflictives dins de cada cromosoma, permetent comparar el risc relatiu de discrepància entre laboratoris de manera normalitzada. S’observa que, excepte el cromosoma mitocondrial (MT), que presenta una taxa considerablement superior però amb un nombre reduït de registres, la majoria de cromosomes mostren valors relativament homogenis, situats aproximadament entre el 18% i el 29%.

Aquesta distribució suggereix que el conflicte en la classificació no es concentra de manera marcada en cromosomes específics, sinó que es distribueix de forma bastant uniforme al llarg del genoma. Les diferències observades són moderades i podrien estar influïdes pel volum de variants disponibles en cada cromosoma.

In [ ]:
# Taxa de conflicte per cromosoma
chrom_conflict_rate = (
    df.groupby("CHROM")["CLASS"]
      .mean()
      .sort_values(ascending=False)
)

plt.figure(figsize=(10, 6))
plt.bar(chrom_conflict_rate.index, chrom_conflict_rate.values)
plt.ylabel("Proportion of CLASS=1")
plt.title("Taxa de conflicte per cromosoma")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


### Distribució de la Posició Genòmica (POS)

L’histograma de la variable **POS** mostra que la distribució de variants al llarg del genoma no és uniforme, sinó que presenta múltiples pics i regions amb elevada concentració de registres. Aquest comportament suggereix l’existència de zones genòmiques amb major densitat de variants reportades, possiblement associades a gens clínicament rellevants o a regions àmpliament estudiades.

La major concentració observada en rangs baixos de posició pot estar relacionada amb la longitud variable dels cromosomes i amb el fet que **POS** representa coordenades internes a cada cromosoma. 

En conjunt, la variable **POS** reflecteix l’heterogeneïtat espacial de les variants en el genoma, però per si sola no permet inferir directament el risc de conflicte sense considerar el context cromosòmic o funcional.

In [ ]:
# Distribucio de POS
plt.figure(figsize=(6, 4))
plt.hist(df["POS"], bins=80)
plt.title("Distribució de la posició genòmica (POS)")
plt.xlabel("Posició genòmica")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

### Distribució dels Al·lels de Referència (REF) i Alternatius (ALT)

L’anàlisi de les freqüències dels valors més comuns de **REF** i **ALT** confirma que la gran majoria de variants corresponen a substitucions simples de nucleòtids (A, C, G i T), és a dir, *Single Nucleotide Variants (SNVs)*. Les combinacions de múltiples nucleòtids, associades a insercions o delecions curtes, representen una proporció molt reduïda del conjunt de dades.

S’observa una lleugera asimetria entre els nucleòtids de referència i els alternatius, amb una major presència de C i G en **REF** i de T i A en **ALT**. En conjunt, aquesta distribució indica que el problema de classificació es concentra principalment en variants puntuals.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# REF
ref_counts = df["REF"].value_counts().head(10)
axes[0].barh(ref_counts.index[::-1], ref_counts.values[::-1])
axes[0].set_title("Top 10 REF allels")
axes[0].set_xlabel("Count")
axes[0].set_ylabel("REF")

# ALT
alt_counts = df["ALT"].value_counts().head(10)
axes[1].barh(alt_counts.index[::-1], alt_counts.values[::-1])
axes[1].set_title("Top 10 ALT allels")
axes[1].set_xlabel("Count")
axes[1].set_ylabel("ALT")

plt.tight_layout()
plt.show()


### Distribució de les Freqüències

Els histogrames mostren que les tres variables presenten una distribució fortament asimètrica cap a la dreta, amb una elevada concentració de valors molt propers a zero. Aquest comportament és coherent amb la naturalesa del dataset, ja que la majoria de variants clíniques són rares en la població general. La forta asimetria suggereix que en fases posteriors podria ser convenient aplicar transformacions com ara escala logarítmica o discretització per reduir l’efecte de la cua llarga.

### Comparació per Classe

Els boxplots segmentats per `CLASS` indiquen que la mediana de les freqüències al·lèliques és pràcticament nul·la en ambdues classes, fet que confirma que la majoria de variants són rares independentment del seu estat de conflicte. Tanmateix, s’observa una lleugera diferència en la dispersió i en la distribució de valors no nuls, suggerint que les variants conflictives tendeixen a concentrar-se en rangs de baixa freqüència però no extremadament rars. Aquest patró és coherent amb la pràctica clínica, on les variants molt comunes solen estar clarament classificades com a benignes, mentre que variants rares poden generar més incertesa i discrepàncies entre laboratoris.

In [ ]:
# Frequencies alleliques
af_cols = [c for c in ["AF_ESP", "AF_EXAC", "AF_TGP"] if c in df.columns]

# Convertim a numeric i assignem NaN quan no es pot convertir
for c in af_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Histogrames
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, col in enumerate(af_cols):
    axes[i].hist(df[col].dropna(), bins=60)
    axes[i].set_title(f"{col} distribució")
    axes[i].set_xlabel("Allele Freqüència")
    axes[i].set_ylabel("Count")

plt.tight_layout()
plt.show()

# Diagrames de caixa per classe
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, col in enumerate(af_cols):
    data0 = df.loc[df["CLASS"] == 0, col].dropna()
    data1 = df.loc[df["CLASS"] == 1, col].dropna()
    axes[i].boxplot([data0, data1], labels=["0", "1"], showfliers=False)
    axes[i].set_title(f"{col} per CLASS")
    axes[i].set_xlabel("CLASS")
    axes[i].set_ylabel("Allele Freqüència")

plt.tight_layout()
plt.show()



## Correlació de Variables Numèriques amb CLASS

La matriu de correlació permet analitzar les relacions lineals entre les variables numèriques del dataset, incloent la variable objectiu **CLASS**. Les correlacions observades amb **CLASS** són generalment baixes en valor absolut, fet que indica que no existeix un únic predictor numèric amb forta relació lineal amb la presència de conflicte. Això suggereix que el patró predictiu és probablement multivariable i pot incloure interaccions no lineals entre característiques.

### Observacions Relevants de la Matriu

S’observa una alta correlació positiva entre **CADD_PHRED** i **CADD_RAW**, fet esperable ja que ambdues mesures provenen del mateix sistema d’anotació funcional. De manera similar, les tres freqüències al·lèliques presenten correlacions elevades entre si, atès que descriuen la mateixa propietat en diferents cohorts poblacionals. En canvi, la relació entre aquestes variables i **CLASS** és negativa però moderada, la qual cosa reforça la idea que les variants més freqüents tendeixen a presentar menys conflicte. Globalment, l’absència de correlacions lineals fortes amb la variable objectiu orienta cap a l’ús de models capaços de capturar relacions no lineals i interaccions complexes entre característiques.

In [ ]:
# Matriu de correlacio (variables numeriques)
num_df = df.select_dtypes(include=[np.number]).copy()
corr = num_df.corr()
sns.set_style("white")  

# Mascara per ocultar el triangle superior
mask = np.triu(np.ones_like(corr, dtype=bool))

plt.figure(figsize=(12, 10))
sns.heatmap(
    corr,
    mask=mask,
    cmap="coolwarm",
    center=0,
    vmin=-1, vmax=1,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)
plt.title("Matriu de correlació", fontsize=14)
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()



In [ ]:
class_corr.head(15)


## Variables categòriques i taxa de conflicte (CLASS)

Els gràfics mostren, per a cada categoria d’una variable, la proporció de variants conflictives (**CLASS=1**) mitjançant les barres blaves, i el nombre total de registres disponibles mitjançant la línia taronja.

La interpretació correcta requereix considerar simultàniament ambdues dimensions: una taxa de conflicte elevada en una categoria amb pocs casos pot ser deguda a soroll estadístic, mentre que diferències observades en categories amb molts registres són més robustes i fiables.

En general, es detecta que algunes variables funcionals (com **IMPACT**, **SIFT** o **PolyPhen**) mostren variacions moderades en la taxa de conflicte entre categories. Això suggereix que el tipus d’anotació funcional podria estar relacionat amb la probabilitat de desacord entre laboratoris. 

En variables d’alta cardinalitat com **SYMBOL**, les diferències s’han d’interpretar amb cautela, ja que poden reflectir el volum de submissions o la complexitat clínica associada a determinats gens més que un efecte biològic directe.

En conjunt, aquests resultats indiquen que una part del senyal predictiu es troba en variables categòriques, especialment quan les diferències de taxa es donen en categories amb volum de dades suficient.

In [ ]:
sns.set_style("white")

def plot_conflict_rate_cat(ax, df, col, target="CLASS", top_n=15):
    tmp = df[[col, target]].copy()
    tmp[col] = tmp[col].astype("object").fillna("NaN")

    # Si hi ha massa categories, agrupem les no principals a "Other"
    vc = tmp[col].value_counts()
    if vc.shape[0] > top_n:
        top = vc.head(top_n).index
        tmp[col] = np.where(tmp[col].isin(top), tmp[col], "Other")

    rate = tmp.groupby(col)[target].mean().sort_values(ascending=False)
    cnt = tmp[col].value_counts().reindex(rate.index)

    ax.bar(rate.index.astype(str), rate.values, color="#4C78A8")
    ax.set_title(f"{col}: taxa de conflicte")
    ax.set_ylabel("Prop. CLASS=1")
    ax.set_xlabel(col)
    ax.tick_params(axis="x", rotation=45, labelsize=8)

    # Segon eix per veure volum de mostres per categoria
    ax2 = ax.twinx()
    ax2.plot(range(len(cnt)), cnt.values, marker="o", linestyle="--", color="#F58518", linewidth=1.5)
    ax2.set_ylabel("Count")

# 6 grafics en graella 2x3
vars_to_plot = [
    ("CLNVC", 10),
    ("IMPACT", 10),
    ("SIFT", 10),
    ("PolyPhen", 10),
    ("BAM_EDIT", 10),
    ("SYMBOL", 20),
]

available = [(c, n) for c, n in vars_to_plot if c in df.columns]

fig, axes = plt.subplots(3, 2, figsize=(22, 10))
axes = axes.flatten()

for i, (col, top_n) in enumerate(available[:6]):
    plot_conflict_rate_cat(axes[i], df, col, top_n=top_n)

# Si en falten, amaguem els subplots buits
for j in range(len(available[:6]), 6):
    axes[j].axis("off")

fig.suptitle("Conflict rate per variables categòriques", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()



## Particio de dades (train/validation/test)

Es defineix una particio fixa per garantir comparacions justes entre models:
- 90% per **train_val** i 10% per **test**
- del **train_val**, 10% per **validation**
- mateixa **random_state** en tots els splits

In [ ]:
# Particio de dades fixa per a tots els models
from sklearn.model_selection import train_test_split

TARGET_COL = "CLASS"
RANDOM_STATE = 42
TEST_SIZE = 0.10
VAL_SIZE_WITHIN_TRAIN = 0.10

X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Primera particio: 90% train_val i 10% test (estratificat)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y,
)

# Segona particio: del train_val, 10% per validacio (estratificat)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val,
    y_train_val,
    test_size=VAL_SIZE_WITHIN_TRAIN,
    random_state=RANDOM_STATE,
    stratify=y_train_val,
)

